Let's just assume we have a trained model and use functions to evaluate it

In [67]:
import pickle
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [68]:
def split_dataset(X: pd.DataFrame, y: pd.DataFrame, number: int, random_state=45) -> tuple:
    """
    This function splits the dataset into the train data and the test data

    Input: 
        - X: training features
        - y: training targets
        - number: the ratio of test samples
    Output: 
        - X_train: training features
        - X_val: test/validation features
        - y_train: training targets
        - y_val: test/validation targets
    """

    X_train = []
    X_val = []
    y_train = []
    y_val = []
    try:
        # Split data into train/test split
        # Add code here
        X_train, X_val, y_train, y_val = train_test_split(
            X,
            y,
            test_size=number/100,
            random_state=random_state
        )
        
        train_percentage = len(X_train)/(len(X_train)+len(X_val))*100
        test_percentage = len(X_val)/(len(X_train)+len(X_val))*100

        # Print dataset split result
        print(
            'The training dataset contains {0:.2f} observations ({1:.2f}%) and the test dataset \
                contains {2:.2f} observations ({3:.2f}%).'.format(
                    len(X_train),
                    train_percentage,
                    len(X_val),
                    test_percentage
                )
        )
    except:
        print('Exception thrown; testing test size to 0')
    return X_train, X_val, y_train, y_val

In [69]:
# Why would we want to inspect the coefficients? Idk, why not, I guess

def inspect_coefficients(models: dict, inspect_models: list) -> dict:
    """
    This function gets the coefficients of the trained models

    Assumes the following:
        - Mult Lin Reg model is of type LinearRegression
        - Poly Reg model is of type Pipeline
        - Ridge Reg model is of type Pipeline and includes cross validation
        - Lasso Reg model is of type Pipeline and includes cross validation

    Input: 
        - models: all trained models
        - inspect_models: the models to be inspected on
    Output: 
        - out_dict: a dicionary contains the coefficients of the selected models, with the following keys:
            - 'Multiple Linear Regression'
            - 'Polynomial Regression'
            - 'Ridge Regression'
            - 'Lasso Regression'
    """
    out_dict = {'Multiple Linear Regression': [],
                'Polynomial Regression': [],
                'Ridge Regression': [],
                'Lasso Regression': []}
    
    # Add code here

    for model in inspect_models:
        if model == "Multiple Linear Regression":
            out_dict[model] = models[model].coef_

        elif model == "Polynomial Regression":
            out_dict[model] = models[model][-1].coef_

        elif model == "Ridge Regression":
            out_dict[model] = models[model][-1].best_estimator_.coef_

        elif model == "Lasso Regression":
            out_dict[model] = models[model][-1].best_estimator_.coef_

    return out_dict

In [70]:
def mae(y_true, y_pred):
    """
    Measures the absolute difference between predicted and actual values

    Input:
        - y_true: true targets
        - y_pred: predicted targets
    Output:
        - mean absolute error
    """
    # Add code here
    mae_score = mean_absolute_error(y_true, y_pred)
    return mae_score

def rmse(y_true, y_pred):
    """
    This function computes the root mean squared error. 
    Measures the difference between predicted and 
    actual values using Euclidean distance.

    Input:
        - y_true: true targets
        - y_pred: predicted targets
    Output:
        - root mean squared error
    """
    # Add code here
    rmse_score = mean_squared_error(y_true, y_pred, squared=False)
    return rmse_score

def r2(y_true, y_pred):
    """
    Compute Coefficient of determination (R2 score). 
    Rrepresents proportion of variance in predicted values 
    that can be explained by the input features.

    Input:
        - y_true: true targets
        - y_pred: predicted targets
    Output:
        - r2 score
    """
    r2_s = r2_score(y_true, y_pred)
    return r2_s

In [71]:
METRICS_MAP = {
    'mean_absolute_error': mae,
    'root_mean_squared_error': rmse,
    'r2_score': r2
}

# Checkpoint 9
def compute_eval_metrics(
        X: pd.DataFrame,
        y_true: pd.DataFrame,
        model,
        metrics: List[str]
) -> dict:
    """
    This function checks the metrics of the models

    Input:
        - X: pandas dataframe with training features
        - y_true: pandas dataframe with true targets
        - model: the model to evaluate
        - metrics: the metrics to evlauate performance 
    Output:
        - metric_dict: a dictionary contains the computed metrics of the selected model, with the following structure:
            - {metric1: value1, metric2: value2, ...}
    """
    metric_dict = {}

    # Add code here
    y_pred = model.predict(X)

    for metric in metrics:
        metric_func = METRICS_MAP.get(metric)

        if metric_func:
            metric_dict[metric] = metric_func(y_true, y_pred)

    return metric_dict

In [72]:
def plot_learning_curve(
        X_train: pd.DataFrame,
        X_val: pd.DataFrame,
        y_train: pd.DataFrame,
        y_val: pd.DataFrame,
        trained_model,
        metrics: list,
        model_name: str
):
    """
    This function plots the learning curve. Note that the learning curve is calculated using 
    increasing sizes of the training samples
    Input:
        - X_train: training features
        - X_val: validation/test features
        - y_train: training targets
        - y_val: validation/test targets
        - trained_model: the trained model to be calculated learning curve on
        - metrics: a list of metrics to be computed
        - model_name: the name of the model being checked
    Output:
        - fig: the plotted figure
        - df: a dataframe containing the train and validation errors, with the following keys:
            - df[metric_fn.__name__ + " Training Set"] = train_errors
            - df[metric_fn.__name__ + " Validation Set"] = val_errors
    """
    fig = make_subplots(
        rows=len(metrics),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=metrics,
    )

    train_size_start = 10
    training_step_size = 5

    df = pd.DataFrame()

    train_errors = {i: [] for i in metrics}
    val_errors = {i: [] for i in metrics}

    # Add code here
    for m in range(train_size_start, len(X_train)+1, training_step_size):
        trained_model.fit(X_train[:m], y_train[:m])

        train_metrics = compute_eval_metrics(X_train[:m], y_train[:m], trained_model, metrics)
        val_metrics = compute_eval_metrics(X_val, y_val, trained_model, metrics)

        for i in metrics:
            train_errors[i].append(train_metrics[i])
            val_errors[i].append(val_metrics[i])

    df_dict = {}

    # Need to map the keys from here to match what the tests are looking for
    metric_test_mapping = {
        "mean_absolute_error": "mae",
        "root_mean_squared_error": "rmse",
        "r2_score": "r2"
    }

    for i in metrics:
        df_dict["{} Training Set".format(metric_test_mapping[i])] = train_errors[i]
        df_dict["{} Validation Set".format(metric_test_mapping[i])] = val_errors[i]

    x = list(range(train_size_start, len(X_train)+1, training_step_size))

    for i in range(len(metrics)):
        metric_name = metrics[i]
        # Need to use go.scatter and fig.append_trace, specifying the row and col of the new trace

        fig.append_trace(
            go.Scatter(
                x=x,
                y=train_errors[metric_name],
                mode="lines+markers",
                marker={"color": "red", "symbol": "cross"},
                name="train"
            ),
            row=i+1,
            col=1
        )

        fig.append_trace(
            go.Scatter(
                x=x,
                y=val_errors[metric_name],
                mode="lines+markers",
                marker={"color": "blue"},
                name="val"
            ),
            row=i+1,
            col=1
        )

        fig.update_xaxes(title_text="Training set size", row=i+1, col=1)
        fig.update_yaxes(title_text=metric_name, row=i+1, col=1)
    
    fig.update_layout(title_text="Training vs val errors for model {}".format(model_name), width=1000,height=1000)


    df = pd.DataFrame(df_dict)

    return fig, df

In [73]:
def create_dataset(features: List[str], split_size: int = 25) -> tuple:
    """
    Create the X_train, X_val, y_train, y_val datasets

    Args:
        features (List[str]): List of features in the dataset we want to train and predict on
        split_size (int): The val dataset size. Converted to a percentage if >1. Default is 25%

    Returns:
        tuple: X_train, X_val, y_train, y_val
    """
    data_df = pd.read_csv("../data/processed_data/feature_data.csv")

    X = data_df[features]
    y = data_df["rat count"]

    return split_dataset(X, y, 25)


In [74]:
def train_models(
        models: List[str],
        X: np.array,
        y: np.array,
        poly_degree: int = 2,
        poly_include_bias: bool = True,
        ridge_cv_fold: int = 3,
        lasso_cv_fold: int = 3
) -> dict:
    """
    Train the models we want for this

    Args:
        models (List[str]): List of models we want to train. Limited to those implemented

    Returns:
        dict: Dict of model_name: trained_model pairs
    """
    trained_models = {}

    model_choices = [
        "Multiple Linear Regression",
        "Polynomial Regression",
        "Ridge Regression",
        "Lasso Regression"
    ]

    for i in models:
        if i not in model_choices:
            print(f"Model {i} is not a valid choice")
            continue

        if i=="Multiple Linear Regression":
            linreg = LinearRegression()
            linreg.fit(X, y)
            trained_models[i] = linreg

        elif i=="Polynomial Regression":
            poly_reg_model = Pipeline(
                [
                    ('scaler', StandardScaler()),
                    ('poly', PolynomialFeatures(
                        degree=poly_degree,
                        include_bias=poly_include_bias
                    )),
                    ('polyReg', LinearRegression())
                ]
            )
            poly_reg_model.fit(X, y)
            trained_models[i] = poly_reg_model

        elif i=="Ridge Regression":
            # Hyperparameters for us to assign
            ridge_solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
            ridge_alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
            ridge_params = {
                'solver': ridge_solvers,
                'alpha': ridge_alphas
            }

            ridge_cv = Pipeline(
                [
                    ('scaler', StandardScaler()),
                    ('ridgeCV', GridSearchCV(
                        estimator=Ridge(),
                        param_grid=ridge_params,
                        cv=ridge_cv_fold
                    ))
                ]
            )
            ridge_cv.fit(X, y)
            trained_models[i] = ridge_cv

        elif i=="Lasso Regression":
            # Hyperparameters for us to assign
            lasso_tol = [0.001, 0.0001]
            lasso_alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
            lasso_params = {
                'tol': lasso_tol,
                'alpha': lasso_alphas
            }

            lasso_cv = Pipeline([
                ('scaler', StandardScaler()),
                ('lassoCV', GridSearchCV(
                    estimator=Lasso(),
                    param_grid=lasso_params,
                    cv=lasso_cv_fold
                ))
            ])
            lasso_cv.fit(X, y)
            trained_models[i] = lasso_cv

    return trained_models

In [75]:
def evaluate_models(
        models: dict,
        metrics: List[str],
        X_train: np.array,
        X_val: np.array,
        y_train: np.array,
        y_val: np.array
) -> None:
    """
    Evaluate the trained models

    Args:
        x

    Returns:
        x
    """
    coefs = inspect_coefficients(models, list(models.keys()))
    print("=====Coefficients=====")
    print(coefs)
    print()

    print("=====Eval metrics=====")
    for model_name, trained_model in models.items():
        metric_select = ['mean_absolute_error', 'root_mean_squared_error', 'r2_score']
        train_metrics = compute_eval_metrics(X_train, y_train, trained_model, metric_select)
        val_metrics = compute_eval_metrics(X_val, y_val, trained_model, metric_select)
        print(f"---{model_name}---")
        print("Train metrics:")
        print(train_metrics)
        print("Val metrics:")
        print(val_metrics)
        print()

        fig, df = plot_learning_curve(
            X_train,
            X_val,
            y_train,
            y_val,
            trained_model,
            metric_select,
            model_name
        )

        fig.show()

In [76]:
# Select features for training and create X and y
features = [
    "population",
    "avg score",
    "critical flag",
    "sidewalk dimensions (area)",
    "roadway dimensions (area)",
    "approved for sidewalk seating",
    "approved for roadway seating",
    "qualify alcohol",
    "total_number_restaurants"
]

# Create dataset
X_train, X_val, y_train, y_val = create_dataset(features)

# Train model(s)
models = train_models(
    [
        "Multiple Linear Regression",
        "Polynomial Regression",
        "Ridge Regression",
        "Lasso Regression",
    ],
    X_train,
    y_train,
    poly_degree=2,
    poly_include_bias=True,
    ridge_cv_fold=5,
    lasso_cv_fold=5
)

# Evaluate model
metric_select = ['mean_absolute_error', 'root_mean_squared_error', 'r2_score']

evaluate_models(models, metric_select, X_train, X_val, y_train, y_val)



The training dataset contains 129.00 observations (75.00%) and the test dataset                 contains 43.00 observations (25.00%).


c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_m

=====Coefficients=====
{'Multiple Linear Regression': array([ 3.28326331e-03, -3.75635859e+00, -4.87762270e-02,  4.89508318e-05,
       -9.78214722e-03,  3.17070701e+00,  1.23911024e+00, -9.50246249e-02,
       -1.05853083e+00]), 'Polynomial Regression': array([ 1.95029707e+15,  1.27341787e+02, -1.12836046e+02,  9.34108417e+02,
        2.78037771e+01, -1.02065893e+03, -7.11359682e+02,  1.40153439e+03,
        4.49057669e+02, -9.97367424e+02,  4.34886625e+01, -2.39323010e+02,
        1.27018410e+02,  2.94617788e+02, -3.65355640e+02, -2.53307333e+02,
       -3.46951625e+02,  7.87282462e+02, -1.86389219e+02,  1.06489227e+01,
       -1.07997735e+03,  7.92567546e+02, -1.28264348e+03, -2.06043540e+03,
        2.54631258e+03, -9.77901957e+02,  2.01578384e+03,  3.08786214e+03,
       -2.30217880e+03,  1.80008798e+03,  9.08074613e+03, -2.52323990e+03,
       -4.10736785e+03, -8.65330341e+03, -2.36644497e+02, -5.70590759e+02,
       -9.02451007e+02,  1.30534862e+03,  6.50462850e+02,  2.66958722e

---Polynomial Regression---
Train metrics:
{'mean_absolute_error': 130.34108527131784, 'root_mean_squared_error': 171.67420864745142, 'r2_score': -0.12640978409783}
Val metrics:
{'mean_absolute_error': 272.6453488372093, 'root_mean_squared_error': 595.4499459076266, 'r2_score': -19.310745915266253}



---Ridge Regression---
Train metrics:
{'mean_absolute_error': 90.45502752293842, 'root_mean_squared_error': 130.67101317716458, 'r2_score': 0.34740337593352355}
Val metrics:
{'mean_absolute_error': 69.96680556892983, 'root_mean_squared_error': 94.6348556376632, 'r2_score': 0.4869760964441825}



c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_m

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.935e+03, tolerance: 2.911e+02

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.422e+02, tolerance: 2.759e+02

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.396e+03, tolerance: 2.623e+02

c:\U

---Lasso Regression---
Train metrics:
{'mean_absolute_error': 92.75214140055589, 'root_mean_squared_error': 135.44939220474757, 'r2_score': 0.2988023901975595}
Val metrics:
{'mean_absolute_error': 68.42629908631483, 'root_mean_squared_error': 93.86171797671882, 'r2_score': 0.4953243502139595}



c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.682e+05, tolerance: 1.074e+03

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.705e+05, tolerance: 1.013e+03

c:\Users\matth\Documents\GitHub\rat-city\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.530e+04, tolerance: 8.134e+02

c:\U

In [77]:
#TODO: Save off a model to a pickle file so that we can test with it
model_to_save = models["Lasso Regression"]
pickle.dump(model_to_save, open("./trained_model.pickle", "wb"))

In [78]:
# Test the pickled file
trained_model = pickle.load(open("./trained_model.pickle", "rb"))

trained_model.predict(
    pd.DataFrame(
        [[62426, 520, 70, 10000, 850, 11, 0, 15, 70]],
        columns=features
    )
)

array([181.2393023])

In [79]:
pd.DataFrame(
        [[62426, 520, 70, 10000, 850, 11, 0, 15, 70]],
        columns=features
    )

,population,avg score,critical flag,sidewalk dimensions (area),roadway dimensions (area),approved for sidewalk seating,approved for roadway seating,qualify alcohol,total_number_restaurants
0,62426,520,70,10000,850,11,0,15,70
